In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error

In [2]:
# df = pd.read_csv('train.csv', thousands = '.')
df = pd.read_csv('train.csv')
df

,일시,광진구,동대문구,성동구,중랑구
0,20180101,0.592,0.368,0.580,0.162
1,20180102,0.840,0.614,1.034,0.260
2,20180103,0.828,0.576,0.952,0.288
3,20180104,0.792,0.542,0.914,0.292
4,20180105,0.818,0.602,0.994,0.308
...,...,...,...,...,...
1456,20211227,3.830,3.416,2.908,2.350
1457,20211228,4.510,3.890,3.714,2.700
1458,20211229,4.490,3.524,3.660,2.524
1459,20211230,4.444,3.574,3.530,2.506


In [3]:
print(df[df['일시'] == 20190907])

           일시  광진구  동대문구  성동구  중랑구
614  20190907  0.0   0.0  0.0  0.0


In [4]:
print(df[df['일시'] == 20190906])
print(df[df['일시'] == 20190908])

           일시   광진구   동대문구    성동구    중랑구
613  20190906  2.02  1.396  1.504  1.024
           일시   광진구   동대문구    성동구    중랑구
615  20190908  5.83  2.658  4.188  2.482


In [5]:
df.loc[614] = [20190907, 3.93, 2.027, 2.846, 1.753]
print(df[613:616])

           일시   광진구   동대문구    성동구    중랑구
613  20190906  2.02  1.396  1.504  1.024
614  20190907  3.93  2.027  2.846  1.753
615  20190908  5.83  2.658  4.188  2.482


In [6]:
def get_outlier(df=None, column=None, weight=1.5):
      # target 값과 상관관계가 높은 열을 우선적으로 진행
  quantile_15 = np.percentile(df[column].values, 25)
  quantile_85 = np.percentile(df[column].values, 75)

  IQR = quantile_85 - quantile_15
  IQR_weight = IQR*weight
  
  lowest = quantile_15 - IQR_weight
  highest = quantile_85 + IQR_weight
  
  outlier_idx = df[column][ (df[column] < lowest) | (df[column] > highest) ].index
  return outlier_idx

In [7]:
columns = ['일시',   '광진구',   '동대문구',    '성동구',    '중랑구']

for i in columns:
    outlier_idx = get_outlier(df=df, column=i, weight=1.5)
    df.drop(outlier_idx, axis=0, inplace=True)

In [8]:
data = df.drop(['광진구','동대문구','성동구','중랑구'], axis=1).values
target = df['광진구'].values

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=1)

In [10]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state=1)
rf.fit(X_train, y_train)

RandomForestRegressor(random_state=1)

In [11]:
print(rf.score(X_train,y_train))
print(rf.score(X_test,y_test))

0.8940347586094715
0.71012282124526


※ RandomForest Regressor : 0.985589536075713

In [12]:
# ['date','Gwangjin-gu','sum','temp','wind speed','hum','supplementary','sunshine duration','insolation']
pred = rf.predict(X_test)
mae = mean_absolute_error(y_test,pred)
print(f'MAE : {mae:.3f}')

MAE : 1.250


In [13]:
sub = pd.read_csv('sample_submission.csv')
print(sub)

           일시  광진구  동대문구  성동구  중랑구
0    20220101    0     0    0    0
1    20220102    0     0    0    0
2    20220103    0     0    0    0
3    20220104    0     0    0    0
4    20220105    0     0    0    0
..        ...  ...   ...  ...  ...
329  20221126    0     0    0    0
330  20221127    0     0    0    0
331  20221128    0     0    0    0
332  20221129    0     0    0    0
333  20221130    0     0    0    0

[334 rows x 5 columns]


In [59]:
test = sub['일시']
test = test.values.reshape(-1,1)


pred = rf.predict(test)
sub['광진구'] = pred
sub


,일시,광진구,동대문구,성동구,중랑구
0,20220101,3.879373,0,0,0
1,20220102,3.879373,0,0,0
2,20220103,3.879373,0,0,0
3,20220104,3.879373,0,0,0
4,20220105,3.879373,0,0,0
...,...,...,...,...,...
329,20221126,3.879373,0,0,0
330,20221127,3.879373,0,0,0
331,20221128,3.879373,0,0,0
332,20221129,3.879373,0,0,0
